In [3]:
host_ip = "127.12.88.05"  # random_value for the moment


In [ ]:
import socket as sk

# let's first code the client's side
import sys
try:
	#create an AF_INET, STREAM socket (TCP)
	s = sk.socket(sk.AF_INET, sk.SOCK_STREAM)
except sk.error:
	print ('Failed to create socket')
	sys.exit()

print ('Socket Created')

In [ ]:
import os
file_name = "file"
file_size = os.path.getsize(file_name)


In [ ]:
start_code = 's'
data_code = 'd'
delimiter = "|"
ack_code = 'a'

def create_start_packet(seq_num:int, file_name:str, file_size:int) -> str:
    global delimiter
    return delimiter.join([start_code, str(seq_num), file_name, str(file_size)]).encode()

client_time_out = 1000 # 1 second

def verify_ack_start(data_received:bytes):
    """This function verifies whether the data sent by the server verifies the following format:
        a|next_seq_num|buff_size where:
        * a is literally the letter acknowledgment string
        * next_seq_num
        """
    data_str = data_received.decode()
    n = data_str.split(delimiter)
    if n < 3 : 
        return None
    comps = data_str.split(delimiter)
    verify = False
    try:
        verify =  comps[0] == ack_code and comps[1].isdigit() and comps[2].isdigit()
    except:
        verify = False
    return comps[:3] if verify else None

client_time_out_count = 5

def send_start_packet(socket_obj: sk.socket, server_add:tuple, seq_num:int, file_name:str, file_size:int):
    # send 5 times with a timeout of 0.5 seconds
    for _ in range(client_time_out_count):
        socket_obj.sendto(create_start_packet(seq_num, file_name, file_size)) 
        socket_obj.timeout(client_time_out)
        server_reply = s.recv()
        # if the server_reply object in not None then the  
        if server_reply : # if the socket received a packet
            reply = verify_ack_start(server_reply) # if the packet falls under the conditions of the server
            if reply is not None: # return if the message is good
                return reply 
        
            



In [ ]:
def create_data_packet(seq_num: int, data:bytes):
    data_str = data.decode()
    return delimiter.join([data_code, str(seq_num), data_str]).encode()

def verify_ack_data(data_received:bytes):
    data_str = data_received.decode()
    n = data_str.split(delimiter)
    if n < 2 : 
        return None
    comps = data_str.split(delimiter)
    verify = False
    try:
        verify =  comps[0] == ack_code and comps[1].isdigit() 
    except:
        verify = False
    return comps[:2] if verify else None



def send_data_packet(socket_obj, server_add:tuple, seq_num:int, data:bytes):
    for _ in range(client_time_out_count):
        socket_obj.sendto(create_data_packet(seq_num, file_name, file_size)) 
        socket_obj.timeout(client_time_out)
        server_reply = s.recv()
        # if the server_reply object in not None then the  
        if server_reply : # if the socket received a packet
            reply = verify_ack_data(server_reply) # if the packet falls under the conditions of the server
            if reply is not None: # return if the message is good
                return reply # return a list containing the components of the server's reply


In [1]:
with open("test_file.txt", 'r') as f:
   print(len(f.read()))
   print(os.path.getsize(os.path.join("test_file.txt")))     

895
895


In [ ]:
## client's code

# first thing, extract arguments from the command line
import sys
import os
from math import floor, ceil
args = sys.argv
assert len(args) == 4  # make sure the correct number of arguments is passed

port = args[1]
file = os.path.join(args[2])
file_name_server = args[3]


try:
	#create an AF_INET, STREAM socket (TCP)
	client_s = sk.socket(sk.AF_INET, sk.SOCK_DGRAM)
except sk.error:
	print ('Failed to create socket')
	sys.exit();
# client socket created
print ('Socket Created')

# get the host address

remote_ip = '127.00.50.34'
# prepare the information needed for starting the session

server_addr = (remote_ip, port)
seq_num = 0
file_size = os.path.getsize(file)

server_reply = send_start_packet(client_s, server_addr, seq_num, file_size)

if server_reply is None:
    print("The server is unavailable!!")
    sys.exit()

a , next_seq, buffer_size = server_reply
next_seq = int(next_seq)
buffer_size = int(buffer_size)

udp_max_header = 16 # an udp packet takes at most 16 bytes
data_trans = buffer_size - udp_max_header
n_trans = ceil( file_size  / data_trans) # the number of transfers
                         


In [ ]:
with open(file, 'r') as f: # this will guarantee the file will be closed regardless of the exectution of the program
    for i in range(n_trans):
        data_bytes = file.read()[i * data_trans: min((i + 1) * data_trans,file_size)] # take the i-th chunk of data
        server_reply = send_data_packet(client_s, server_addr, next_seq, data_bytes)
        if server_reply is None:
            print("The server is unavailable")
            sys.exit()
        else:
            next_seq = int(server_reply[1])
